In [1]:
import skimage.io as io
import time
import numpy as np
import cv2
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from ChainCodesFeatures import *
from PolygonFeatures import * 
from WindowFeatures import *
from windows import *

In [16]:
def Feature_Extractor(img):
    
    st = time.time()
    edged = cv2.Canny(img, 30, 200)
    contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    en = time.time()
#     print("Contours : ",(en-st)*1000,"ms")
    
    st = time.time()
    windows, clusters, clusteredWindows = getClusteredWindows2(img,13,10)
    en = time.time()
#     print("Kesho : ",(en-st)*1000,"ms")
    
    st = time.time()
    F1,F2,F3,F4,F5,F7,F8,F9 = Chain_Codes_Feature_Extractor(img,contours,clusteredWindows)
    en = time.time()
#     print("Ramzy : ",(en-st)*1000,"ms")
    
    st = time.time()
    f10, f11, f12, f13, f14 = PolygonFeatures(contours)
    en = time.time()
#     print("Ibrahim Lefta : ",(en-st)*1000,"ms")
    
    st = time.time()
    Windows = list()
    for i in range(clusteredWindows.shape[0]):
        Windows  += list(WindowFeatures(clusteredWindows[i].astype(np.uint8)))
        
    en = time.time()
#     print("October : ",(en-st)*1000,"ms")
    
    Features = list(F1)+list(F2)+list(F3)+list(F4)+list(F5)+list(F7)+list(F8)+list(F9)+list(f10)+list(f11)+list(f12)+list(f13)+list(f14)+Windows
    return Features
    

In [3]:
img = cv2.imread("TestCases/class1_0.png", cv2.IMREAD_GRAYSCALE)
Features_1 = Feature_Extractor(img)
img = cv2.imread("TestCases/class0_0.png", cv2.IMREAD_GRAYSCALE)
Features_0 = Feature_Extractor(img)
img = cv2.imread("TestCases/class1_1.png", cv2.IMREAD_GRAYSCALE)
Features_pred_1 = Feature_Extractor(img)
img = cv2.imread("TestCases/class0_1.png", cv2.IMREAD_GRAYSCALE)
Features_pred_0 = Feature_Extractor(img)

In [4]:
features_test = np.array([Features_0,Features_1])
features_train = np.array([Features_pred_0,Features_pred_1])

In [5]:
gnb = GaussianNB()
gnb.fit(features_train,[0,1])

GaussianNB(priors=None, var_smoothing=1e-09)

In [6]:
result = gnb.predict(features_test)
actual_label = [0,1]
Acc = (actual_label==result).sum()/len(actual_label)
print("Accuracy : ",Acc*100,"%")

Accuracy :  100.0 %


In [17]:
with open("forms.txt") as h:
    lines = h.readlines()[16:]

In [18]:
freq = np.zeros(680, dtype=int)
formWriter = {}
for i in lines:
    line = i.split(' ')
    key = line[1]
    if key in formWriter.keys():
        formWriter[key].append(line[0])
    else:
        formWriter[key] = [line[0]]

In [19]:
freq = [x for x, l in formWriter.items() if len(l)>2]

In [20]:
acc = 0
total = 0
for itr in range(100):
    keys = np.random.randint(len(freq), size=3)
    train = []
    test = []
    for k in keys:
        forms = formWriter[freq[k]]
        forms = [freq[k]+'-'+'{:03d}'.format(s)+'.jpg' for s in range(len(forms))]
#         print(forms)
        idxes = np.random.randint(len(forms), size=3)
        
        train.append(forms[idxes[0]])
        train.append(forms[idxes[1]])
        test.append(forms[idxes[2]])
    testWriter = np.random.randint(len(test))
    testImg = test[testWriter]
    
    trainFeatures = []
    Y = []
    for i in range(len(train)):
        img = cv2.imread('train/'+train[i], cv2.IMREAD_GRAYSCALE)
        if img is None:
            continue
        f = Feature_Extractor(img)
        trainFeatures.append(f)
        Y.append(i//2)
    gnb = SVC()
    gnb.fit(trainFeatures, Y)
    
    img = cv2.imread("train/"+testImg, cv2.IMREAD_GRAYSCALE)
    testFeatures = Feature_Extractor(img)                     
    result = gnb.predict([testFeatures])
    if result[0] == testWriter:
        acc += 1
    total +=1
    print(total,acc, "Accuracy : ",acc*100/total,"%")
    print(result, testWriter)

1 1 Accuracy :  100.0 %
[0] 0
2 2 Accuracy :  100.0 %
[2] 2
3 2 Accuracy :  66.66666666666667 %
[2] 0
4 3 Accuracy :  75.0 %
[0] 0
5 4 Accuracy :  80.0 %
[2] 2
6 5 Accuracy :  83.33333333333333 %
[0] 0
7 6 Accuracy :  85.71428571428571 %
[0] 0
8 7 Accuracy :  87.5 %
[0] 0
9 8 Accuracy :  88.88888888888889 %
[2] 2
10 9 Accuracy :  90.0 %
[0] 0
11 10 Accuracy :  90.9090909090909 %
[2] 2
12 11 Accuracy :  91.66666666666667 %
[0] 0
13 12 Accuracy :  92.3076923076923 %
[2] 2
14 13 Accuracy :  92.85714285714286 %
[2] 2
15 14 Accuracy :  93.33333333333333 %
[1] 1
16 15 Accuracy :  93.75 %
[0] 0
17 15 Accuracy :  88.23529411764706 %
[2] 0
18 15 Accuracy :  83.33333333333333 %
[0] 1
19 16 Accuracy :  84.21052631578948 %
[2] 2
20 17 Accuracy :  85.0 %
[0] 0
21 17 Accuracy :  80.95238095238095 %
[1] 0
22 18 Accuracy :  81.81818181818181 %
[0] 0
23 19 Accuracy :  82.6086956521739 %
[2] 2
24 20 Accuracy :  83.33333333333333 %
[1] 1
25 21 Accuracy :  84.0 %
[2] 2
26 22 Accuracy :  84.61538461538461 

In [24]:
acc

79

In [10]:
itr

32